# Recognize named entities on Twitter with LSTMs

In this assignment, you will use a recurrent neural network to solve Named Entity Recognition (NER) problem. NER is a common task in natural language processing systems. It serves for extraction such entities from the text as persons, organizations, locations, etc. In this task you will experiment to recognize named entities from Twitter.

For example, we want to extract persons' and organizations' names from the text. Than for the input text:

    Ian Goodfellow works for Google Brain

a NER model needs to provide the following sequence of tags:

    B-PER I-PER    O     O   B-ORG  I-ORG

Where *B-* and *I-* prefixes stand for the beginning and inside of the entity, while *O* stands for out of tag or no tag. Markup with the prefix scheme is called *BIO markup*. This markup is introduced for distinguishing of consequent entities with similar types.

A solution of the task will be based on neural networks, particularly, on Bi-Directional Long Short-Term Memory Networks (Bi-LSTMs).

### Libraries

For this task you will need the following libraries:
 - [pytorch](https://pytorch.org/) — an open-source software library for Machine Intelligence.
    
 - [Numpy](http://www.numpy.org) — a package for scientific computing.
 

### Data

The following cell will download all data required for this assignment into the folder `./data`.

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    ! wget https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py -O setup_google_colab.py
    import setup_google_colab
    setup_google_colab.setup_week2()

import sys
sys.path.append("..")
from common.download_utils import download_week2_resources

download_week2_resources()

### Load the Twitter Named Entity Recognition corpus

We will work with a corpus, which contains tweets with NE tags. Every line of a file contains a pair of a token (word/punctuation symbol) and a tag, separated by a whitespace. Different tweets are separated by an empty line.

The function *read_data* reads a corpus from the *file_path* and returns two lists: one with tokens and one with the corresponding tags. You need to complete this function by adding a code, which will replace a user's nickname to `<USR>` token and any URL to `<URL>` token. You could think that a URL and a nickname are just strings which start with *http://* or *https://* in case of URLs and a *@* symbol for nicknames.

In [ ]:
def read_data(file_path):
    tokens = []
    tags = []
    
    tweet_tokens = []
    tweet_tags = []
    for line in open(file_path, encoding='utf-8'):
        line = line.strip()
        if not line:
            if tweet_tokens:
                tokens.append(tweet_tokens)
                tags.append(tweet_tags)
            tweet_tokens = []
            tweet_tags = []
        else:
            token, tag = line.split()
            # Replace all urls with <URL> token
            # Replace all users with <USR> token

            ######################################
            ######### YOUR CODE HERE #############
            ######################################
                
            tweet_tokens.append(token)
            tweet_tags.append(tag)
            
    return tokens, tags

And now we can load three separate parts of the dataset:
 - *train* data for training the model;
 - *validation* data for evaluation and hyperparameters tuning;
 - *test* data for final evaluation of the model.

In [ ]:
train_tokens, train_tags = read_data('data/train.txt')
validation_tokens, validation_tags = read_data('data/validation.txt')
test_tokens, test_tags = read_data('data/test.txt')

You should always understand what kind of data you deal with. For this purpose, you can print the data running the following cell:

In [ ]:
for i in range(2):
    for token, tag in zip(train_tokens[i], train_tags[i]):
        print('%s\t%s' % (token, tag))
    print()

### Prepare dictionaries

To train a neural network, we will use two mappings: 
- {token}$\to${token id}: address the row in embeddings matrix for the current token;
- {tag}$\to${tag id}: one-hot ground truth probability distribution vectors for computing the loss at the output of the network.

Now you need to implement the function *build_dict* which will return {token or tag}$\to${index} and vice versa. 

In [ ]:
from collections import defaultdict
import numpy as np

In [ ]:
def build_dict(tokens_or_tags, special_tokens):
    """
        tokens_or_tags: a list of lists of tokens or tags
        special_tokens: some special tokens
    """
    # Create a dictionary with default value 0
    # The lambda function gets called whenever it needs a default value.

    tok2idx = defaultdict(lambda: 0)
    idx2tok = []
    
    # Create mappings from tokens (or tags) to indices and vice versa.
    # At first, add special tokens (or tags) to the dictionaries.
    # The first special token must have index 0.
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    
    # Mapping tok2idx should contain each token or tag only once. 
    # To do so, you should:
    # 1. extract unique tokens/tags from the tokens_or_tags variable, which is not
    #    occur in special_tokens (because they could have non-empty intersection)
    # 2. index them (for example, you can add them into the list idx2tok
    # 3. for each token/tag save the index into tok2idx).
    
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    return tok2idx, idx2tok

After implementing the function *build_dict* you can make dictionaries for tokens and tags. Special tokens in our case will be:
 - `<UNK>` token for out of vocabulary tokens;
 - `<PAD>` token for padding sentence to the same length when we create batches of sentences.

In [ ]:
special_tokens = ['<UNK>', '<PAD>']
special_tags = ['O']

# Create dictionaries 
token2idx, idx2token = build_dict(train_tokens + validation_tokens, special_tokens)
tag2idx, idx2tag = build_dict(train_tags, special_tags)

The next additional functions will help you to create the mapping between tokens and ids for a sentence. 

In [ ]:
def words2idxs(tokens_list):
    return [token2idx[word] for word in tokens_list]

def tags2idxs(tags_list):
    return [tag2idx[tag] for tag in tags_list]

def idxs2words(idxs):
    return [idx2token[idx] for idx in idxs]

def idxs2tags(idxs):
    return [idx2tag[idx] for idx in idxs]

## Build a recurrent neural network

This is the most important part of the assignment. Here we will specify the network architecture based on TensorFlow building blocks. It's fun and easy as a lego constructor! We will create an LSTM network which will produce probability distribution over tags for each token in a sentence. To take into account both right and left contexts of the token, we will use Bi-Directional LSTM (Bi-LSTM). Dense layer will be used on top to perform tag classification.  

In [ ]:
import torch
from torch.nn import LSTM
from torch import nn
from torch import optim
import torch.nn.functional as F

[This page](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html) may help you understand the following code.

[This page](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html) may help you to understand the following cell.

In [ ]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = ######### YOUR CODE HERE #############


        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = ######### YOUR CODE HERE #############

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

# Train

In [ ]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

Create LSTMTagger model with the following parameters:
* embedding_dim — dimension of embeddings, recommended value: 200, 
* hidden_dim — size of hidden layers for RNN, recommended value: 200;
* vocab_size — number of tokens;
* tagset_size — number of tags;

In [ ]:
model = ######### YOUR CODE HERE #############
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

See what the scores are before training
Note that element i,j of the output is the score for tag j for word i.
Here we don't need to train, so the code is wrapped in torch.no_grad()


In [ ]:
with torch.no_grad():
    inputs = torch.tensor(words2idxs(train_tokens[0]))
    tag_scores = model(inputs)

    print(tag_scores[6:9])
    print(tags2idxs(train_tags[0])[6:9])

tensor([[-1.9308, -1.5112, -1.8668, -2.0909, -2.1039, -1.4519],
        [-1.9598, -1.4916, -1.9578, -2.1166, -2.0857, -1.3932],
        [-1.9082, -1.5217, -1.9122, -2.1426, -2.1020, -1.4025]])
[0, 1, 2]


### Simplified Train

In [ ]:
for epoch in range(300): 
    for sentence, tags in zip(train_tokens[:2], train_tags[:2]):
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = ######### YOUR CODE HERE #############
        targets = ######### YOUR CODE HERE #############

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = ######### YOUR CODE HERE #############
        loss.backward()
        optimizer.step()

In [ ]:
# See what the scores are after training
with torch.no_grad():
    inputs = torch.tensor(words2idxs(train_tokens[0]))
    tag_scores = model(inputs)

    print(train_tokens[0][6:9])
    print(tag_scores.shape)
    for score in tag_scores.numpy()[6:9]:
      print([ "%.2f" % s for s in score])

    print(tags2idxs(train_tags[0])[6:9])


['for', 'Ghostland', 'Observatory']
torch.Size([26, 6])
['-0.22', '-4.20', '-3.95', '-4.31', '-2.08', '-3.69']
['-1.05', '-2.06', '-2.63', '-2.73', '-1.32', '-2.15']
['-0.92', '-2.27', '-1.97', '-2.82', '-1.92', '-1.89']
[0, 1, 2]


# TODOs


* Plot training loss curve
* Mini-batch processing
  * check dimensions
  * Padding to the max length
* Loss function
  * Cross entropy loss
  * Applying mask to the loss 
* Dynamic learning rate
* Evaluate on val/test set by Precision, Recall, F1 metrics
* Hyperparameter tuning:
  * EMBEDDING_DIM
  * HIDDEN_DIM
  * batch size
  * number of epochs
  * ...
* Apply BiLSTM
* Try Adam optimizer






### Generate batches

Neural Networks are usually trained with batches. It means that weight updates of the network are based on several sequences at every single time. The tricky part is that all sequences within a batch need to have the same length. So we will pad them with a special `<PAD>` token. It is also a good practice to provide RNN with sequence lengths, so it can skip computations for padding parts. We provide the batching function *batches_generator* readily available for you to save time. 

In [ ]:
def batches_generator(batch_size, tokens, tags,
                      shuffle=True, allow_smaller_last_batch=True):
    """Generates padded batches of tokens and tags."""
    print("batch size: ", batch_size)
    print("tokens len: ", len(tokens))
    print("tags len: ", len(tags))

    n_samples = len(tokens)
    if shuffle:
        order = np.random.permutation(n_samples)
    else:
        order = np.arange(n_samples)

    n_batches = n_samples // batch_size
    if allow_smaller_last_batch and n_samples % batch_size:
        n_batches += 1

    for k in range(n_batches):
        batch_start = k * batch_size
        batch_end = min((k + 1) * batch_size, n_samples)
        current_batch_size = batch_end - batch_start
        x_list = []
        y_list = []
        max_len_token = 0
        for idx in order[batch_start: batch_end]:
            x_list.append(words2idxs(tokens[idx]))
            y_list.append(tags2idxs(tags[idx]))
            max_len_token = max(max_len_token, len(tags[idx]))
            
        # Fill in the data into numpy nd-arrays filled with padding indices.
        x = np.ones([current_batch_size, max_len_token], dtype=np.int32) * token2idx['<PAD>']
        y = np.ones([current_batch_size, max_len_token], dtype=np.int32) * tag2idx['O']
        lengths = np.zeros(current_batch_size, dtype=np.int32)
        for n in range(current_batch_size):
            utt_len = len(x_list[n])
            x[n, :utt_len] = x_list[n]
            lengths[n] = utt_len
            y[n, :utt_len] = y_list[n]
        yield x, y, lengths